## Sample N "valid" vowels

In [1]:
from os import path
import functools
import itertools

from evoclearn.core.samplers import Uniform
from evoclearn.core.utils import KeepLastValue
from evoclearn.core.io import load_bounds, wav_write
from evoclearn.core import mappings, filters, constraints
from evoclearn.core import vocaltractlab as vtl

In [2]:
etcdir = path.join("..", "etc")
vtl.initialise(path.join(etcdir, "JD2_speaker.xml"))

<CDLL '/home/demitasse/workspace/evoclearn/evl_core/evoclearn/core/VocalTractLabApi.so', handle 3f2dcf0 at 0x7fce70156f70>

In [3]:
speaker_bounds = load_bounds(open(path.join(etcdir, "JD2_bounds.json")))
all_params = list(speaker_bounds)
neutral_values = [speaker_bounds[p]["default"] for p in all_params]
neutral_seq = seq = next(Uniform(speaker_bounds, labels=["V"]))
neutral_seq.loc[:, all_params] = neutral_values

sampled_params = ["HX", "HY", "JA", "JX", "LD", "LP", "TBX", "TBY", "TCX", "TCY", "TS1", "TS2", "TS3", "TTX", "TTY", "VO", "VS"]

## Implement the random sampling loop for V segments

In [4]:
v_duration = [8800 / 44100] # approximately 0.2s

#### Set up the simulation pipeline:

In [5]:
def sample_with_defaults(seq):
  full_seq = neutral_seq.copy()
  full_seq.loc[(0 ,"V"), sampled_params] = list(seq[sampled_params].iloc[0])
  return full_seq
add_slope_and_duration = functools.partial(mappings.add_slope_and_duration, durations=v_duration)

sampler = Uniform({k:speaker_bounds[k] for k in sampled_params}, labels=["V"], seed=777)
full_samples = map(sample_with_defaults, sampler)
valid_samples = KeepLastValue(filters.discard_by_constraints(full_samples, constraints=[constraints.vowel_voiced]))
qta_targets = map(add_slope_and_duration, valid_samples)
vtl_curves = map(mappings.target_approximation, qta_targets)
speech_sounds = map(mappings.synthesise_vtl_curves, vtl_curves)

#### Define the simulation loop with stopping criteria, and save updates to files as needed:

In [6]:
twenty_iterations = itertools.islice(speech_sounds, 20)

for i, wav in enumerate(twenty_iterations):
    print(i)
    wav_write(wav, f"iteration_{str(i).zfill(5)}.wav")
    with open(f"iteration_{str(i).zfill(5)}.targets.json", "w") as outf:
      outf.write(valid_samples.last_value.to_json())


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
